In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

False 2.2.0+cpu


In [1]:
def decor(f):
    def wrapper():
        f()
        print('aaa')
    return wrapper

class MyClass:
    def __init__(self):
        # 定义动态方法
        self.decor = decor
        setattr(self, 'abc', self.decor(self.abc))

    def abc(self):
        print("This is method abc")

class MyClass2(MyClass):
    def abc(self):
        print("This is method abc2")

# 创建类实例
obj = MyClass2()

# 调用动态方法
obj.abc()



This is method abc2
aaa


In [1]:
# %% test a specific model
from src.interface import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)

24-05-21 21:24:53|MOD:display     |: Model Specifics:
24-05-21 21:24:53|MOD:display     |: Start Process [Data] at Tue May 21 21:24:53 2024!


--Process Queue : Data + Fit + Test
--Model_name is set to gru_lgbm_ShortTest!
Callback : DynamicDataLink() , assign and unlink dynamic data in tra networks
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=10) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=10,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=10) , display batch progress bar
Callback : StatusDisplay(verbosity=10) , display epoch and event information
{'random_seed': None,
 'model_name': 'gru_lgbm_ShortTest',
 'model_mod

24-05-21 21:24:56|MOD:display     |: Finish Process [Data], Cost 3.0 Secs
24-05-21 21:24:56|MOD:display     |: Start Process [Fit] at Tue May 21 21:24:56 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


  0%|          | 0/10 [00:00<?, ?it/s]

score function of [spearman] calculated and success!
loss function of [pearson] calculated and success!


Valid Ep#  0 score : 0.06673: 100%|██████████| 10/10 [00:01<00:00,  8.60it/s]
FirstBite Ep#  0 : loss  0.99628, train 0.00685, valid 0.00858, best 0.0086, lr1.3e-03
Valid Ep#  1 score : -0.02383: 100%|██████████| 10/10 [00:01<00:00,  8.51it/s]
FirstBite Ep#  1 : loss  0.97932, train 0.02392, valid 0.03608, best 0.0361, lr2.5e-03
Valid Ep#  2 score : -0.13844: 100%|██████████| 10/10 [00:01<00:00,  8.13it/s]
FirstBite Ep#  2 : loss  0.95643, train 0.04112, valid 0.05681, best 0.0568, lr3.8e-03
Valid Ep#  3 score : 0.02695: 100%|██████████| 10/10 [00:01<00:00,  8.66it/s]
FirstBite Ep#  3 : loss  0.94138, train 0.05820, valid 0.05188, best 0.0568, lr5.0e-03
24-05-21 21:26:08|MOD:display     |: gru_lgbm_ShortTest #0 @20170103|FirstBite Ep#  4 Max Epoch|Train 0.0582 Valid 0.0519 BestVal 0.0568|Cost  1.2Min, 14.0Sec/Ep
24-05-21 21:26:08|MOD:display     |: Finish Process [Fit], Cost 0.0 Hours, 1.2 Min/model, 18.0 Sec/Epoch
24-05-21 21:26:08|MOD:display     |: Start Process [Test] at Tue May 21

                    hook_name  num_calls  total_time   avg_time
10             on_train_batch         40   37.146631   0.928666
21           on_fit_model_end          1   28.109833  28.109833
27              on_test_batch        105   15.773787   0.150227
15        on_validation_batch         40    4.661100   0.116528
4          on_fit_model_start          1    2.073456   2.073456
24        on_test_model_start          1    0.645275   0.645275
28          on_test_batch_end        105    0.141632   0.001349
25   on_test_model_type_start          3    0.055852   0.018617
17    on_validation_epoch_end          4    0.038894   0.009724
11         on_train_batch_end         40    0.034968   0.000874
16    on_validation_batch_end         40    0.031930   0.000798
31                on_test_end          1    0.019947   0.019947
29     on_test_model_type_end          3    0.007977   0.002659
12         on_train_epoch_end          4    0.006020   0.001505
6        on_train_epoch_start          4

In [ ]:
from src.interface import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)


In [ ]:
# prepare data
from src.interface import DataModule
DataModule.prepare_data()